# 논문 구현 
- Model of Price Optimization as a Part of Hotel Revenue Management—Stochastic Approach 를 읽고 구현하였음 
- 기본 idea : 각 판매경로별 수요탄력성을 구하여, 월 별로 가격에 반응하는 정도를 기반으로 적절한 가격을 책정 
- method : scipy의 optimize.minimize, SLSQP 알고리즘 

In [10]:

import os
import numpy as np
from numpy.random import default_rng
import itertools
from scipy.stats import truncnorm 
import statistics
import scipy as sp
import math
import pandas as pd
from tqdm import tqdm
import itertools
from itertools import permutations

os.chdir(r"C:\submit")
df=pd.read_csv(r".\input\microdata.csv") 

### 수요탄력성 PART

In [ ]:
dfmean=dfpart.groupby(['판매구분2','월']).mean().reset_index()
dfmean=dfmean[['판매구분2','월','입장객']]
dfmean.loc[dfmean['월'].isin([1,2,7,8,10,12]),'입장객']+=100
dfmean.loc[~dfmean['월'].isin([1,2,7,8,10,12]),'입장객']-=50


#직관적인 truncnorm 함수 
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

epdtotal={}

partlist=sorted(list(set(dfpart['판매구분2'])))
months=list(range(1,13))
years=[2015,2016,2017,2018,2019,2022]


num_simulations =1000 # 예를 들어 1000번 


for way, month in tqdm(list(itertools.product(partlist, months))):
    epd1 = []
    pq=dfpart
    pq=pq[(pq['판매구분2']== way)&(pq['월']== month)]
    if len(pq)==0:
        continue
    else:
        pq = pq[['할인율', '입장객']]
        mean=dfmean.loc[(dfmean['판매구분2']==way)&(dfmean['월']==month),'입장객'].item()
        pq['입장객']=pq['입장객']-mean
        pq.loc[pq['입장객']<0,'입장객']=0

        pq.reset_index(drop=True, inplace=True)
        # pq.drop(pq[pq['입장객']==0].index,inplace=True) # 삭제하지 않으니까 q의 최빈값이 0으로 df에 들어가는 문제 발생
        pq.reset_index(drop=True, inplace=True)
        
        # word는 segment (일단 경로) 
        # j는 monte carlo 시뮬레이션 반복하는 중 
        # k는 segment 내에서 한 가격에 실제 경우의 수 길이
        if pq.shape[0]==0:
            epdtotal[f'{way},{month}']=np.nan
            continue
        else: 
            """# p의 dictionary"""
            pdict = list(set(pq['할인율']))

            for j in range(num_simulations):
        
                for p1 , p2 in list(permutations(pdict,2)):
                    
                    if p1 == p2 : 
                        continue

                    else: # p1 != p2의 경우에 대해 

                        q1 = pq.loc[pq['할인율']==p1, '입장객'] # p1에 대한 실제 입장객 수 
                        q1.reset_index(drop=True,inplace=True)
                        q2 = pq.loc[pq['할인율']==p2, '입장객'] # p2에 대한 실제 입장객 수로 분포 구해서 난수 생성 후 무작위 추출 활용
                        q2.reset_index(drop=True,inplace=True)
                        q1.mean()
                        q2.mean()
                            # q2 난수 생성, q2의 mode 값을 df로 갖는 카이제곱분포, q1과 연산 위해 q1 길이만큼 추출
                            
                        
                        X = get_truncated_normal(mean=np.mean(q2), sd=4, low=np.mean(q2)-np.std(q2), upp=np.mean(q2)+np.std(q2)+1) 
                        qd2 = X.rvs(len(q1),random_state=602)

                        for k in range(len(q1)): # q1 길이만큼 돌아감
                            epd = (qd2[k] - q1[k]) / (q1[k] + qd2[k]) / (p1 - p2) * (2- p2 - p1)
                            epd1.append(epd) 
            epdtotal[f'{way},{month}']=epd1




In [12]:
#"""만들어진 epdtotal dic 저장(계속 만들기 너무 번거로움)"""
import pickle
os.chdir(r"C:\optimization\optim_validation")
with open("epdtotal_del_month_effect.pkl","wb") as f:
    pickle.dump(epdtotal, f)
#불러오기
with open("epdtotal_del_month_effect.pkl","rb") as f:
    epdtotal = pickle.load(f)

### 최적화 PART

In [139]:


def by_year_optim_rate(dfpart,year,month,gundang_jehu,prices,susuryo,predicted_visitor):
    totalelasticity_byrandom=pd.DataFrame()
    percentdic_dataframe=pd.DataFrame()
    
    dfpart=dfpart
    years=year
    months=month
    gundang_jehu=gundang_jehu
    prices=prices

    fun=[]

    sorted_partlist=sorted(list(set(dfpart['판매구분2'])))
    sorted_partlist.remove('통신사전회원')

    p = [np.round(np.mean(dfpart.loc[(dfpart['판매구분2']==sorted_partlist[i])& \
                            # (dfpart['년']==year) & 
                            (dfpart['월']==month),
                            '할인율']),2) \
                                for i in range(len(sorted_partlist))]
    p = pd.Series(p, dtype=object).fillna(0).tolist()


    q = predicted_visitor


    price=np.array([prices]*len(p))
    pp=np.array([0]*len(p))
    gundang=[0,#간편결제
            -(price*(1-pp)*susuryo[0])[0], #소셜_상시
            -(price*(1-pp)*susuryo[1])[0],#소셜_특가
            -(price*(1-pp)*susuryo[2])[0],#우대_네이버
            0, #우대_카드동반
            0, #우대_특별
            0,#카드본인전회원
            gundang_jehu, #카드본인제휴
            0,#통신사KT
            # 0,#통신사전회원
            0,#통신사제휴
            0] # 통신사기타 

    # 목적함수


    def f1(pp,*args):
        p=args[0]
        q=args[1]
        epdrandom=args[2]
        gundang=args[3]
        price=args[4]

        p=np.array(p)
        q=np.array(q)
        epdrandom=np.array(epdrandom)
        gundang=np.array(gundang)
        price=np.array(price)

        return - ((q + epdrandom * (price *(1-pp) - price *(1-p)) * q /(price * (1-p))) \
            * (price * (1-pp)) + gundang).sum() 
    
    """제약조건"""        
    def eq_constraints0(pp,*args): #카드제휴의 할인율은 0.5으로 고정  (등식 제약조건 )
        return pp[7]-0.50

    def ieq_constraints0(pp,*args):#카드전회원>소셜 (부등식 제약조건 )
        return pp[6]-pp[2]

    cons=(
        {'type':'eq','fun': eq_constraints0},
        {'type':'ineq','fun': ieq_constraints0}
            )

    """상하한선"""
    bnds=tuple([[0.3,0.61]])*len(p)

    num_simulations = 1000
    percentdic={}
    epdlist={}

    for way in sorted_partlist:
        percentdic[f'{way},{year},{month}']=[]
        epdlist[f'{way},{year},{month}']=[]

    for j in tqdm(range(num_simulations)):

        # segment에 대해 epd를 monte carlo 시뮬레이터, 앞서 구한 epd의 분포로 난수 생성
        
        epdrandom = [ 0 if epdtotal[f'{way},{month}']==[]
                        else np.random.normal(np.mean(epdtotal[f'{way},{month}']),4) 
                        for way in sorted_partlist ] 
        
        for num, way in enumerate(sorted_partlist):
            epdlist[f'{way},{year},{month}'].append(epdrandom[num])
        
        ppd1 = sp.optimize.minimize(f1,
                                    np.array([0.4]*len(p)), 
                                    args=tuple([p,q,epdrandom,gundang,price]),
                                    method='SLSQP',
                                    bounds=bnds,
                                    options={'disp': 0 ,'maxiter':1000},
                                    constraints=cons
                                    )


        if ppd1['status']==0:
            fun.append(ppd1['fun'])
            for num, way in enumerate(sorted_partlist):
                percentdic[f'{way},{year},{month}'].append(ppd1['x'][num])

        else: continue

    print("{}년 {}월".format(year,month))

    for way in sorted_partlist:
        print(f"{way}:{np.round(np.mean(percentdic[f'{way},{year},{month}']),2)}")


In [67]:
sorted_partlist=sorted(list(set(dfpart['판매구분2'])))
sorted_partlist.remove('통신사전회원')
sorted_partlist


['간편결제',
 '소셜_상시',
 '소셜_특가',
 '우대_네이버',
 '우대_카드동반',
 '우대_특별',
 '카드본인전회원',
 '카드본인제휴',
 '통신사KT',
 '통신사SKT제휴',
 '통신사기타']

In [135]:

"""input 예시"""

gundang_jehu=12345 #카드제휴 건당보전금
prices=100000 #정가
susuryo= [0.011,0.22,0.033] # 소셜 수수료 [상시, 특가, 네이버]
year=2022 #예측 년
month=4 #예측 월
predicted_visitor=[1000, #간편결제 예상입장객
                   1000, #소셜상시 예상입장객
                   1000, #소셜특가
                   1000, #우대네이버
                   1000, #우대카드동반
                   1000, #우대특별
                   1000, #카드본인전회원
                   1000, #카드본인제휴
                   1000, #통신사kt
                   1000, #통신사skt제휴
                   1000]# 통신사기타

In [140]:
"""여기를 돌리면 결과가 프린트됨"""
np.random.seed(102)
by_year_optim_rate(dfpart=dfpart,
                   year=2022,
                   month=12,
                   gundang_jehu=12345,
                   prices=100000,
                   susuryo=[0.011,0.22,0.033],
                   predicted_visitor=predicted_visitor)

100%|██████████| 1000/1000 [00:23<00:00, 42.95it/s]

2022년 12월
간편결제:0.56
소셜_상시:0.44
소셜_특가:0.45
우대_네이버:0.44
우대_카드동반:0.51
우대_특별:0.44
카드본인전회원:0.53
카드본인제휴:0.5
통신사KT:0.52
통신사SKT제휴:0.54
통신사기타:0.51
